Requires a .env environment with a username and password with the following format:
    CLIENT_USER={username here, no brackets}
    CLIENT_PASS={password here, no brackets}

In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as cond

from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

import undetected_chromedriver as uc


from dotenv import load_dotenv
from pathlib import Path

import pandas as pd
import os
import regex as re
import hashlib

path = f'{os.getcwd()}/.env'
load_dotenv(dotenv_path = path)

ModuleNotFoundError: No module named 'undetected_chromedriver'

In [4]:
'''
    Some stuff that I was testing to get around bot detection 

'''

# chrome_options = Options()
# chrome_options.add_argument("user-data-dir=selenium")

# driver.get('https://icollege.gsu.edu')
# handle = driver.current_window_handle
# driver.service.stop()
# time.sleep(6)

# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# driver.switch_to.window(handle)



driver = uc.Chrome(use_subprocess=True)
driver.get('https://icollege.gsu.edu')


USERNAME = os.getenv('CLIENT_USER')
PASSWORD = os.getenv('CLIENT_PASS')



In [7]:
# Section to automatically login using a username * password

WebDriverWait(driver, 10).until(
    cond.element_to_be_clickable(
        (By.XPATH, '//*[@id="ubtn-3743"]')
    )
).click()

# time.sleep(2)
if driver.current_url != 'https://gastate.view.usg.edu/d2l/home':
    WebDriverWait(driver, 5).until(
        cond.element_to_be_clickable(
            (By.XPATH, '//*[@id="loginForm:username"]')
        )
    ).send_keys(USERNAME)

    WebDriverWait(driver, 5).until(
        cond.element_to_be_clickable(
            (By.XPATH, '//*[@id="loginForm:password"]')
        )
    ).send_keys(PASSWORD)

    driver.find_element(By.XPATH, '//*[@id="loginForm:loginButton"]').click()

In [ ]:

# class ID was hard coded into URL, easier to go to specific quiz/assignment section rather than clicking
class_id = {
    2636346: 'DM',
    2636257: 'OS',
    2635988: 'DBS',
    2635485: 'SWE',
}

# stores assignments and quizzes
assigns = {}
quizzes = {}

invalid_keywords = ['Evaluation', 'Category', 'per', 'available', 'Feedback']

for num in class_id.keys():
    assign_url = f'https://gastate.view.usg.edu/d2l/lms/dropbox/user/folders_list.d2l?ou={num}&isprv=0'
    quiz_url = f'https://gastate.view.usg.edu/d2l/lms/quizzing/user/quizzes_list.d2l?ou={num}'
    assigns[num] = []
    quizzes[num] = []

    driver.get(assign_url)
    WebDriverWait(driver, 5).until(
        cond.element_to_be_clickable(
            (By.TAG_NAME, 'tr')
        )
    )

    page_text = driver.find_elements(By.TAG_NAME, 'tr')

    for elements in page_text:
        if any(words for words in invalid_keywords if words in elements.text):
            continue
        
        assigns[num] += [elements.text]

    driver.get(quiz_url)
    WebDriverWait(driver, 5).until(
        cond.element_to_be_clickable(
            (By.TAG_NAME, 'tr')
        )
    )
    page_text = driver.find_elements(By.TAG_NAME, 'tr')
    for elements in page_text:
        if any(words for words in invalid_keywords if words in elements.text):
            continue
        else:
            quizzes[num] += [elements.text]




with open('due dates.txt', 'w+') as output:
    output.seek(0)
    for key in assigns.keys():
        output.write('------------------------------------\n')
        output.write(f'{class_id[key]}\n')
        for x in assigns[key]:
            output.write(x)


        output.write('\n\n')

